In [1]:
# Utilitarios para modificar el esquema de metadatos
from pyspark.sql.types import StructType, StructField

#Importamos los tipos de datos que definiremos para cada campo
from pyspark.sql.types import StringType, IntegerType, DoubleType

#Importamos la librerIa de pandas compatible con entornos de clUster de Big Data
import pyspark.pandas as pd

#Por defecto un dataframe Pandas muestra 1000 registros
#Vamos a indicarle que solo muestre 20 para que no se sature el notebook
pd.set_option("display.max_rows", 20)

#Libreria para manipular los servicios de AWS
import boto3

Calculation started (calculation_id=92c91821-c048-2ef8-9d96-436f0083f9b0) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [3]:
#Nombre del bucket desde donde se lee el archivo
#IMPORTANTE: REEMPLAZAR "XXX" POR TUS INICIALES
bucket = "datasetsbdajac"


Calculation started (calculation_id=48c91822-2783-1f8e-11b6-493481663d31) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [4]:
#Ruta dentro del archivo dentro del bucket
rutaDeArchivo = "data/insurance"

Calculation started (calculation_id=8ec91822-514d-f86a-45a9-e0a3f5f2b475) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [5]:
#Definicion del esquema de metadatos
schema = StructType(
    [
        StructField("age", DoubleType(), True),
        StructField("sex", StringType(), True),
        StructField("bmi", DoubleType(), True),
        StructField("children", DoubleType(), True),
        StructField("smoker", StringType(), True),
        StructField("region", StringType(), True),
        StructField("charges", DoubleType(), True)
    ]
)

Calculation started (calculation_id=5ec91822-7804-0d8f-aaec-68d8a6e719ba) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [6]:
#Definimos las variables categoricas
categorias = [
  "sex",
  "region",
  "smoker"
]

Calculation started (calculation_id=38c91822-9f8f-5f5d-0a66-074a41931282) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [7]:
#Campo label
label = "charges"

Calculation started (calculation_id=c6c91822-ba95-1017-842a-bff70798f84f) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [8]:
#Definimos la ruta desde donde se lee el archivo de datos que se preparara
#En Python 3, podemos anteponer "f" en una cadena para reemplazar valores
rutaArchivoRaw = f"s3://{bucket}/{rutaDeArchivo}/"

Calculation started (calculation_id=6ac91823-2943-90f1-8759-58c3553db3be) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [9]:
#Verificamos
print(rutaArchivoRaw)

Calculation started (calculation_id=5cc91823-4449-ba3c-9214-f63ec9b6772a) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
s3://datasetsbdajac/data/insurance/



In [10]:
#Definimos la ruta en donde se almacenara el 100% de los registros del dataset preparado
rutaArchivoDataset = f"s3://{bucket}/{rutaDeArchivo}_dataset/"

#Verificamos
print(rutaArchivoDataset)

Calculation started (calculation_id=fac91823-795c-3df8-bb50-6429cbc04fe7) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
s3://datasetsbdajac/data/insurance_dataset/



In [11]:
#Leemos los datos
dfRaw = spark.read.format("csv").option("header", "true").option("delimiter", ",").option("encoding", "ISO-8859-1").schema(schema).load(rutaArchivoRaw)

#Vemos el esquema de metadatos
dfRaw.printSchema()

#Verificamos
dfRaw.show()

Calculation started (calculation_id=f0c91824-81c4-2347-e06c-fde3c4ea5646) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
root
 |-- age: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: double (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)

+----+------+------+--------+------+---------+-----------+
| age|   sex|   bmi|children|smoker|   region|    charges|
+----+------+------+--------+------+---------+-----------+
|19.0|female|  27.9|     0.0|   yes|southwest|  16884.924|
|18.0|  male| 33.77|     1.0|    no|southeast|  1725.5523|
|28.0|  male|  33.0|     3.0|    no|southeast|   4449.462|
|33.0|  male|22.705|     0.0|    no|northwest|21984.47061|
|32.0|  male| 28.88|     0.0|    no|northwest|  3866.8552|
|31.0|female| 25.74|     0.0|    no|southeast|  3756.6216|
|46.0|female| 33.44|     1.0|    no|southeast|  8240.5896|
|37.0|female| 27.74|     3.0|    no|northwest|  7281.5056|
|37.0|  male| 29.83|     2.0|    no|northeast|  6406.4107|
|60

In [12]:
#Convertimos el dataframe SPARK a un dataframe PANDAS
dfpRaw = pd.from_pandas(dfRaw.toPandas())

#Verificamos
dfpRaw

Calculation started (calculation_id=06c91824-e67e-3d25-e3ae-b8303652b6e9) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
     age     sex     bmi  children smoker     region      charges
0   19.0  female  27.900       0.0    yes  southwest  16884.92400
1   18.0    male  33.770       1.0     no  southeast   1725.55230
2   28.0    male  33.000       3.0     no  southeast   4449.46200
3   33.0    male  22.705       0.0     no  northwest  21984.47061
4   32.0    male  28.880       0.0     no  northwest   3866.85520
5   31.0  female  25.740       0.0     no  southeast   3756.62160
6   46.0  female  33.440       1.0     no  southeast   8240.58960
7   37.0  female  27.740       3.0     no  northwest   7281.50560
8   37.0    male  29.830       2.0     no  northeast   6406.41070
9   60.0  female  25.840       0.0     no  northwest  28923.13692
10  25.0    male  26.220       0.0     no  northeast   2721.32080
11  62.0  female  26.290       0.0    yes  southeast  27808.72510
12  23.0    male  34.400       0.0     no  southwest   1826.84300
13  56.0  female  39.820       0.0     no  southeast 

In [13]:
#Conversion de variables categoricas a variables numericas en columnas dummy
dfpDataset = pd.get_dummies(dfpRaw, columns = categorias)

#Verificamos
dfpDataset

Calculation started (calculation_id=58c91825-2241-6002-5974-a93e9b4fcbd5) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
     age     bmi  children      charges  sex_female  sex_male  region_northeast  region_northwest  region_southeast  region_southwest  smoker_no  smoker_yes
0   19.0  27.900       0.0  16884.92400           1         0                 0                 0                 0                 1          0           1
1   18.0  33.770       1.0   1725.55230           0         1                 0                 0                 1                 0          1           0
2   28.0  33.000       3.0   4449.46200           0         1                 0                 0                 1                 0          1           0
3   33.0  22.705       0.0  21984.47061           0         1                 0                 1                 0                 0          1           0
4   32.0  28.880       0.0   3866.85520           0         1                 0                 1                 0                 0          1           0
5   31.0  25.740       0.0   3756.6

In [14]:
#Convertimos el dataframe PANDAS a un dataframe SPARK
dfDataset = dfpDataset.to_spark()

#Verificamos
dfDataset.show()

Calculation started (calculation_id=c0c91825-7935-e57d-4caa-b39d752910e5) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+----+------+--------+-----------+----------+--------+----------------+----------------+----------------+----------------+---------+----------+
| age|   bmi|children|    charges|sex_female|sex_male|region_northeast|region_northwest|region_southeast|region_southwest|smoker_no|smoker_yes|
+----+------+--------+-----------+----------+--------+----------------+----------------+----------------+----------------+---------+----------+
|19.0|  27.9|     0.0|  16884.924|         1|       0|               0|               0|               0|               1|        0|         1|
|18.0| 33.77|     1.0|  1725.5523|         0|       1|               0|               0|               1|               0|        1|         0|
|28.0|  33.0|     3.0|   4449.462|         0|       1|               0|               0|               1|               0|        1|         0|
|33.0|22.705|     0.0|21984.47061|         0|       1|               0|               1|               0|        

In [15]:
#Colocamos hasta el final la columna LABEL
dfDatasetOrdenado = dfDataset.select(
    dfDataset["charges"],
    dfDataset["age"],
    dfDataset["bmi"],
    dfDataset["children"],
    dfDataset["sex_female"],
    dfDataset["sex_male"],
    dfDataset["region_northeast"],
    dfDataset["region_northwest"],
    dfDataset["region_southeast"],
    dfDataset["region_southwest"],
    dfDataset["smoker_no"],
    dfDataset["smoker_yes"]
)

#Verificamos
dfDatasetOrdenado.show()

Calculation started (calculation_id=bac91825-e056-1be0-bd0a-650f5c28ad24) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+-----------+----+------+--------+----------+--------+----------------+----------------+----------------+----------------+---------+----------+
|    charges| age|   bmi|children|sex_female|sex_male|region_northeast|region_northwest|region_southeast|region_southwest|smoker_no|smoker_yes|
+-----------+----+------+--------+----------+--------+----------------+----------------+----------------+----------------+---------+----------+
|  16884.924|19.0|  27.9|     0.0|         1|       0|               0|               0|               0|               1|        0|         1|
|  1725.5523|18.0| 33.77|     1.0|         0|       1|               0|               0|               1|               0|        1|         0|
|   4449.462|28.0|  33.0|     3.0|         0|       1|               0|               0|               1|               0|        1|         0|
|21984.47061|33.0|22.705|     0.0|         0|       1|               0|               1|               0|        

In [16]:
#Almacenamos el dataframe que tiene el 100% de registros
dfDatasetOrdenado.write.format("csv").option("header", "false").option("delimiter", ",").option("encoding", "ISO-8859-1").mode("overwrite").save(rutaArchivoDataset)

Calculation started (calculation_id=d2c91826-8f82-9635-5253-8f8cec5d018d) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [17]:
#Nos conectamos al servicio de "S3"
s3 = boto3.client("s3")

Calculation started (calculation_id=90c91827-06b3-c158-7f69-55e25a497e5c) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [18]:
#Eliminamos el archivo "_SUCCESS" del dataset en donde se encuentran el 100% de registros
s3.delete_object(
    Bucket = bucket,
    Key = f"{rutaDeArchivo}_dataset/_SUCCESS"
)

Calculation started (calculation_id=e6c91827-2abe-7ef1-cf8e-59645ebc959c) in (session=3ac91821-7fcf-5492-13d1-7ac10056f63d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
{'ResponseMetadata': {'RequestId': 'HREZRXNXV8YXA4CN', 'HostId': 'Ru4Tl0/qZ0fuOi4Pv55CtqcHjFWmget+Lo4e3A57ls6S1c1RZ61khosDezXB38wYE8oQ3wY9lRcB0OOm7nbM9oubt2EPjrgktqx2icVi4bM=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'Ru4Tl0/qZ0fuOi4Pv55CtqcHjFWmget+Lo4e3A57ls6S1c1RZ61khosDezXB38wYE8oQ3wY9lRcB0OOm7nbM9oubt2EPjrgktqx2icVi4bM=', 'x-amz-request-id': 'HREZRXNXV8YXA4CN', 'date': 'Thu, 26 Sep 2024 21:48:14 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}

